In [41]:
import pandas as pd
import numpy as np
import time
import pandas as pd

In [42]:
# Variaveis de controle que serão ultilizadas no projeto
tolerancias = list() # Lista de tolerancias
omegas = list() # lista de erros maximos permitido
max_iterations = list() # lista de iterações máximas usados
tempos = list() # tempo q o algoritimo levou pra finalizar
metodos = list() # metodo ultilizado
tipo_matriz = list() # tipo de matriz veiculada

W = [1,1.3,1.4,1.50,1.6,1.7,1.8,1.9,2] # valores de omega sugeridos no arquivo
N = [500,1000,5000,10000] # valores de maximas iterações sugeridos no arquivo
TOL = [0.001, 0.0000001] # valores de tolerancia sugeridos no arquivo

In [43]:
# A matriz tridiagonal demanda uma matriz que dimensao nxn / n>=3
while True:
    dim_matriz = int(input("Qual o formato da matriz (>=3)? "))
    if dim_matriz <= 2:
        print("Matriz muito pequena")
    else:
        break

Qual o formato da matriz (>=3)? 4


# Matriz triadiagonal

In [44]:
#Função que gera uma matriz triadiagonal como especificada pelo exercicio

def gerar_matriz(n):
    matriz = [[0] * n for _ in range(n)]  # Inicializa a matriz com zeros
    
    for i in range(n):
        for j in range(n):
            if i == j:
                matriz[i][j] = 8
            elif i < j:
                matriz[i][j] = -2
            elif i > j:
                matriz[i][j] = -1
            if j >= i + 2 or i >= j + 2:
                matriz[i][j] = 0
    
    return matriz

matriz =  gerar_matriz(dim_matriz)
matriz

[[8, -2, 0, 0], [-1, 8, -2, 0], [0, -1, 8, -2], [0, 0, -1, 8]]

In [91]:
# Função que convete a matriz gerada acima numa matriz nx3 como proposto pelo exercicio
def converter_matriz(matriz):
    n = len(matriz)
    matriz_convertida = [[0] * 3 for _ in range(n)]  # Inicializa a matriz convertida com zeros
    vetor_independente = [0] * n  # Inicializa o vetor de termos independentes com zeros

    for i in range(n):
        for j in range(n):
            if i == j:
                matriz_convertida[i][1] = matriz[i][j]
            elif j == i + 1:
                matriz_convertida[i][2] = matriz[i][j]
            elif i == j + 1:
                matriz_convertida[i][0] = matriz[i][j]
            
            vetor_independente[i] += matriz[i][j]

    return matriz_convertida, vetor_independente

#Converter a matriz e gerar um vetor independente
matriz_convertida, vetor = converter_matriz(matriz)
matriz_convertida, vetor = np.array(matriz_convertida), np.array(vetor)
print(matriz_convertida, vetor)

[[ 0  8 -2]
 [-1  8 -2]
 [-1  8 -2]
 [-1  8  0]] [6 5 5 7]


In [92]:
#Função que resolve pelo sistema thomas
def resolver_sistema_thomas(a,b,c,d):  
    inicio = time.time()
    #preliminares  
    a = a.astype('double')  
    b = b.astype('double')  
    c = c.astype('double')  
    d = d.astype('double')  
 
    #recupera ordem do sistema  
    n=np.shape(a)[0]  
 
    #inicializa vetor x  
    x=np.zeros(n)  
 
    #calcula cl e dl sobrescrevendo-os em c e d  
    c[0]=c[0]/b[0]  
    for i in np.arange(1,n-1,1):  
       c[i]=c[i]/(b[i]-a[i]*c[i-1])  
 
    d[0]=d[0]/b[0]  
    for i in np.arange(1,n,1):  
       d[i]=(d[i]-a[i]*d[i-1])/(b[i]-a[i]*c[i-1])  
 
    #faz substituicao reversa para obter a solucao x  
    x[n-1]=d[n-1]  
    for i in np.arange(n-2,-1,-1):  
       x[i]=d[i]-c[i]*x[i+1]
    fim = time.time()
    tempo =  fim - inicio
    tolerancias.append('Nan')
    omegas.append('Nan')
    max_iterations.append('Nan')
    tempos.append(tempo)
    metodos.append('Thomas')
    tipo_matriz.append('Tridiagonal')
    return x  


In [95]:
#Resolução pelo sistema tomas

a=matriz_convertida[:,2] #Diagonal acima da diagonal principal
b=matriz_convertida[:,1] #Diagonal principa
c=matriz_convertida[:,0] #Diagonal abaixo da diagonal principal
d=vetor # Vetor independente 

triadiagonal_thomas = resolver_sistema_thomas(a,b,c,d)
triadiagonal_thomas

# A resolução pelo sistema thomas para a matriz proposta acaba gerando divisão por zero

array([0.75      , 0.93346774, 0.96774194, 0.875     ])

### Avaliando o algoritimo SOR

In [48]:
import numpy as np

def resolver_sistema_sor(matrizsor, vetor_independente, omega, max_iteracoes, tolerancia, metodo, tipomatriz):
    matriz = np.array(matrizsor)
    n = len(matriz)
    x = np.zeros(n)  # Inicializa o vetor solução x com zeros
    inicio = time.time()
    for _ in range(max_iteracoes):
        x_anterior = x.copy()
        
        for i in range(n):
            soma1 = np.dot(matriz[i, :i], x[:i])
            soma2 = np.dot(matriz[i, i + 1:], x_anterior[i + 1:])
            
            x[i] = (1 - omega) * x_anterior[i] + (omega / matriz[i, i]) * (vetor_independente[i] - soma1 - soma2)
        
        erro = np.linalg.norm(x - x_anterior)
        
        if erro < tolerancia:
            break
        fim = time.time()
        tempo =  fim - inicio
        tolerancias.append(tolerancia)
        omegas.append(omega)
        max_iterations.append(max_iteracoes)
        tempos.append(tempo)
        metodos.append(metodo)
        tipo_matriz.append(tipomatriz)
    
    return x

In [49]:
X = resolver_sistema_sor(matrizsor=matriz, vetor_independente=vetor, omega=1, max_iteracoes=10,
                         tolerancia=0.001, metodo='SOR', tipomatriz='Tridiagonal')
X

array([0.99998771, 0.99999598, 0.99999919, 0.9999999 ])

### Tetando o algoritimo SOR combinando varios parâmetros

In [50]:
# estrutura de repetiçõa combinando todas os valores de omega, max iterations e tolerancias
for w in W:
    for n in N:
        for tol in TOL: 
            X = resolver_sistema_sor(matriz, vetor, omega=w, max_iteracoes=n, tolerancia=tol,
                                    metodo='SOR', tipomatriz='Tridiagonal')

# Matriz penta diagonal

In [51]:
# a matriz penta diagonal precisa ter dimensão maior que n=5 no minimo
while True:
    dim_matriz = int(input("Qual o formato da matriz (>=5)? "))
    if dim_matriz <= 5:
        print("Matriz muito pequena")
    else:
        break

Qual o formato da matriz (>=5)? 6


In [52]:
# Função que gera uma matriz penta diagonal no moldes do exercicio
import numpy as np

def gerar_matriz_penta_diagonal(n, q):
    matriz = np.zeros((n, n))
    
    for i in range(n):
        matriz[i, i] = 8  # Diagonal principal
        
        if i < n-1:
            matriz[i+1, i] = -1  # Diagonal abaixo da principal
            
        if i > 0:
            matriz[i-1, i] = -2  # Diagonal acima da principal
        
        if i + q < n:
            matriz[i + q, i] = -1  # Diagonal flutuante abaixo da principal
            
        if i - q >= 0:
            matriz[i - q, i] = -2  # Diagonal flutuante acima da principal
    
    return matriz

def gerar_vetor_independente(matriz):
    return np.sum(matriz, axis=1)

# Gerar matriz penta diagonal
matriz_pentadiagonal = gerar_matriz_penta_diagonal(dim_matriz, 5-1)
print("Matriz Penta Diagonal:")
print(matriz_pentadiagonal)

# Gerar vetor independente
vetor_independente_pentadiagonal = gerar_vetor_independente(matriz_pentadiagonal)
vetor_independente_pentadiagonal = vetor_independente_pentadiagonal.astype(int)
print("\nVetor Independente:")
print(vetor_independente_pentadiagonal)


Matriz Penta Diagonal:
[[ 8. -2.  0.  0. -2.  0.]
 [-1.  8. -2.  0.  0. -2.]
 [ 0. -1.  8. -2.  0.  0.]
 [ 0.  0. -1.  8. -2.  0.]
 [-1.  0.  0. -1.  8. -2.]
 [ 0. -1.  0.  0. -1.  8.]]

Vetor Independente:
[4 3 5 5 4 6]


In [53]:
def extrair_diagonais(matriz):
    n = len(matriz)
    diagonais = np.zeros((n, 5))

    for i in range(n):
        # Diagonal principal
        diagonais[i, 2] = matriz[i, i]

        # Diagonal acima da principal
        if i > 0:
            diagonais[i, 1] = matriz[i, i-1]

        # Diagonal abaixo da principal
        if i < n-1:
            diagonais[i, 3] = matriz[i, i+1]

        # Diagonal acima da diagonal principal
        if i > 1:
            diagonais[i, 0] = matriz[i, i-1]

        # Diagonal abaixo da diagonal principal
        if i < n-2:
            diagonais[i, 4] = matriz[i, i+1]

    return diagonais


matriz_diagonais = extrair_diagonais(matriz_pentadiagonal)
matriz_diagonais = matriz_diagonais.astype(int)

print("\nMatriz de Diagonais:")
print(matriz_diagonais)



Matriz de Diagonais:
[[ 0  0  8 -2 -2]
 [ 0 -1  8 -2 -2]
 [-1 -1  8 -2 -2]
 [-1 -1  8 -2 -2]
 [-1 -1  8 -2  0]
 [-1 -1  8  0  0]]


In [54]:
type(matriz_diagonais)

numpy.ndarray

### Método SOR para matrizes penta diagonais

In [55]:
X = resolver_sistema_sor(matriz_pentadiagonal, vetor_independente_pentadiagonal, 1, 10, 0.001,
                         metodo='SOR', tipomatriz='Pentadiagonal')
X

array([0.99978661, 0.99985766, 0.99990685, 0.99992348, 0.99993709,
       0.99997434])

### Tetando o algoritimo SOR combinando varios parâmetros

In [56]:
# estrutura de repetiçõa combinando todas os valores de omega, max iterations e tolerancias
for w in W:
    for n in N:
        for tol in TOL: 
            X = resolver_sistema_sor(matriz, vetor, omega=w, max_iteracoes=n, tolerancia=tol,
                                    metodo='SOR', tipomatriz='Pentadiagonal')

# INterpretação dos resultado

In [57]:
            
# armazenamento dos resultado num dataframe
data_frame_sor = pd.DataFrame({'Omega': omegas, 'Tolerância': tolerancias, 'Max_iterations': max_iterations,
                               'Tempo': tempos, 'Métodos': metodos, 'Tipomatriz': tipo_matriz})
data_frame_sor

,Omega,Tolerância,Max_iterations,Tempo,Métodos,Tipomatriz
0,Nan,Nan,Nan,0.000000,Thomas,Tridiagonal
1,1,0.001,10,0.000000,SOR,Tridiagonal
2,1,0.001,10,0.000000,SOR,Tridiagonal
3,1,0.001,10,0.000000,SOR,Tridiagonal
4,1,0.001,10,0.000000,SOR,Tridiagonal
...,...,...,...,...,...,...
70903,2,0.0,10000,0.639695,SOR,Pentadiagonal
70904,2,0.0,10000,0.639695,SOR,Pentadiagonal
70905,2,0.0,10000,0.639695,SOR,Pentadiagonal
70906,2,0.0,10000,0.639695,SOR,Pentadiagonal


In [58]:
data_frame_sor = data_frame_sor.drop_duplicates()
data_frame_sor

,Omega,Tolerância,Max_iterations,Tempo,Métodos,Tipomatriz
0,Nan,Nan,Nan,0.000000,Thomas,Tridiagonal
1,1,0.001,10,0.000000,SOR,Tridiagonal
6,1,0.001,500,0.000000,SOR,Tridiagonal
11,1,0.0,500,0.000000,SOR,Tridiagonal
14,1,0.0,500,0.006515,SOR,Tridiagonal
...,...,...,...,...,...,...
70390,2,0.0,10000,0.620188,SOR,Pentadiagonal
70605,2,0.0,10000,0.629198,SOR,Pentadiagonal
70791,2,0.0,10000,0.636257,SOR,Pentadiagonal
70870,2,0.0,10000,0.639264,SOR,Pentadiagonal


In [59]:
data_frame_sor[data_frame_sor["Métodos"]=='Thomas']

,Omega,Tolerância,Max_iterations,Tempo,Métodos,Tipomatriz
0,Nan,Nan,Nan,0.0,Thomas,Tridiagonal


In [90]:
data_frame_sor[(data_frame_sor["Tipomatriz"]=='Tridiagonal')].sort_values(["Tempo"], ascending=False)

,Omega,Tolerância,Max_iterations,Tempo,Métodos,Tipomatriz
35444,2,0.0,10000,0.630475,SOR,Tridiagonal
35380,2,0.0,10000,0.624989,SOR,Tridiagonal
35345,2,0.0,10000,0.621476,SOR,Tridiagonal
35338,2,0.0,10000,0.620548,SOR,Tridiagonal
35337,2,0.0,10000,0.619871,SOR,Tridiagonal
...,...,...,...,...,...,...
822,1.7,0.0,5000,0.000000,SOR,Tridiagonal
873,1.7,0.001,10000,0.000000,SOR,Tridiagonal
899,1.7,0.0,10000,0.000000,SOR,Tridiagonal
950,1.8,0.001,500,0.000000,SOR,Tridiagonal
